In [1]:
#!pip install modin[all]

In [2]:
#import modin.pandas as pd

In [3]:
#import ray
#ray.init()

In [4]:
#%%time
#pd.read_csv('Desktop/week 6/2019-October/2019-Oct.csv')

In [5]:
import pandas as pd #normal pandas

In [6]:
%%time
x=pd.read_csv('Desktop/week 6/2019-October/2019-Oct.csv')

Wall time: 1min 7s


In [7]:
x.head(14)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
6,2019-10-01 00:00:08 UTC,view,17300353,2053013553853497655,NaN,creed,380.96,555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d
7,2019-10-01 00:00:08 UTC,view,31500053,2053013558031024687,NaN,luminarc,41.16,550978835,6280d577-25c8-4147-99a7-abc6048498d6
8,2019-10-01 00:00:10 UTC,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
9,2019-10-01 00:00:11 UTC,view,1004545,2053013555631882655,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb


In [8]:
new= x[['product_id','price','brand','user_id']].copy()

In [9]:
import numpy as np
new['brand'].fillna('unknown', inplace=True)

In [10]:
new.head()

,product_id,price,brand,user_id
0,44600062,35.79,shiseido,541312140
1,3900821,33.20,aqua,554748717
2,17200506,543.10,unknown,519107250
3,1307067,251.74,lenovo,550050854
4,1004237,1081.98,apple,535871217


In [11]:
#new.to_csv('New.csv', index = False, header=True)

In [12]:
%%writefile testutility.py
import logging
import os
import subprocess
import pandas as pd
import yaml
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting testutility.py


In [13]:
%%writefile file.yaml
file_type: csv
dataset_name: newdata
file_name: New
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: {'product_id', 'price', 'brand', 'user_id'}

Overwriting file.yaml


In [14]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [15]:
config_data['columns']

{'product_id': None, 'price': None, 'brand': None, 'user_id': None}

In [16]:
config_data

{'file_type': 'csv',
 'dataset_name': 'newdata',
 'file_name': 'New',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': {'product_id': None,
  'price': None,
  'brand': None,
  'user_id': None}}

In [17]:
#read the file using config file
file_type = config_data['file_type']
source_file = "Desktop/week 6/" + config_data['file_name'] + f'.{file_type}'
#print("",source_file)
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,product_id,price,brand,user_id
0,44600062,35.79,shiseido,541312140
1,3900821,33.20,aqua,554748717
2,17200506,543.10,unknown,519107250
3,1307067,251.74,lenovo,550050854
4,1004237,1081.98,apple,535871217


In [18]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [19]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['product_id', 'price', 'brand', 'user_id'], dtype='object')
columns of YAML are: {'product_id': None, 'price': None, 'brand': None, 'user_id': None}


In [20]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    count_row = df.shape[0]  # Gives number of rows
    print("total number of rows", count_row)
    count_col = df.shape[1] # Gives number of columns
    print("total number of col", count_col)
    #source_file
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed
total number of rows 42448764
total number of col 4
